In [55]:
import requests
import csv 
url="https://quotes.toscrape.com/"
data=requests.get(url)
data.status_code
from bs4 import BeautifulSoup
soup=BeautifulSoup(data.text,"html.parser")
links={
    l.get_text(): f"{url}{l.get('href')}"
    for l in soup.select("div.col-md-4 a")
    if l.get('href') and "/tag/" in l.get('href')
        }
My_quotes=[]
for category,link in links.items() :
    my_link=link
    while my_link:
        data=requests.get(my_link)
        soup=BeautifulSoup(data.text,"html.parser" )
        quotes_div=soup.select("div.quote")
        for quote in quotes_div:
         text=quote.select_one("span.text").get_text()
         author=quote.select_one("small.author").get_text()
         My_quotes.append({
            "category":category,
            "quote":text,
            "author":author,
        
        })

        next_page = soup.select_one("ul.pager li.next a")
        if next_page:
            my_link = url + next_page.get("href")
        else:
            my_link = None


csv_file = "quotes_by_category.csv"

with open(csv_file, mode="w", encoding="utf-8", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["category", "quote", "author"])
    writer.writeheader()
    writer.writerows(My_quotes)

